# EXTRACT

Extract csv from FiveThirtyEight: https://github.com/fivethirtyeight/data/tree/master/candy-power-ranking

Extract csv from USDA: https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/methods-and-application-of-food-composition-laboratory/mafcl-site-pages/sr11-sr28/

In [22]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine


In [2]:
#import USDA data
csv_file = "USDA.csv"
usda_df = pd.read_csv(csv_file, encoding = 'latin1')
usda_df.head()

,NDB_No,Shrt_Desc,Water_(g),Energ_Kcal,Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),Sugar_Tot_(g),...,Vit_K_(µg),FA_Sat_(g),FA_Mono_(g),FA_Poly_(g),Cholestrl_(mg),GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Refuse_Pct
0,1001,"BUTTER,WITH SALT",15.87,717,0.85,81.11,2.11,0.06,0.0,0.06,...,7.0,51.368,21.021,3.043,215.0,5.00,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,0.0
1,1002,"BUTTER,WHIPPED,W/ SALT",16.72,718,0.49,78.30,1.62,2.87,0.0,0.06,...,4.6,45.390,19.874,3.331,225.0,3.80,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,0.0
2,1003,"BUTTER OIL,ANHYDROUS",0.24,876,0.28,99.48,0.00,0.00,0.0,0.00,...,8.6,61.924,28.732,3.694,256.0,12.80,1 tbsp,205.0,1 cup,0.0
3,1004,"CHEESE,BLUE",42.41,353,21.40,28.74,5.11,2.34,0.0,0.50,...,2.4,18.669,7.778,0.800,75.0,28.35,1 oz,17.0,1 cubic inch,0.0
4,1005,"CHEESE,BRICK",41.11,371,23.24,29.68,3.18,2.79,0.0,0.51,...,2.5,18.764,8.598,0.784,94.0,132.00,"1 cup, diced",113.0,"1 cup, shredded",0.0


In [3]:
#Split SHRT_Desc on commas to extract food type
split_df = pd.concat([usda_df['Shrt_Desc'].str.split(',', expand=True), usda_df[['NDB_No', 'Water_(g)', 'Energ_Kcal', 'Protein_(g)', 'Sugar_Tot_(g)']]], axis=1)

split_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,NDB_No,Water_(g),Energ_Kcal,Protein_(g),Sugar_Tot_(g)
0,BUTTER,WITH SALT,None,None,None,None,None,None,None,None,None,None,1001,15.87,717,0.85,0.06
1,BUTTER,WHIPPED,W/ SALT,None,None,None,None,None,None,None,None,None,1002,16.72,718,0.49,0.06
2,BUTTER OIL,ANHYDROUS,None,None,None,None,None,None,None,None,None,None,1003,0.24,876,0.28,0.00
3,CHEESE,BLUE,None,None,None,None,None,None,None,None,None,None,1004,42.41,353,21.40,0.50
4,CHEESE,BRICK,None,None,None,None,None,None,None,None,None,None,1005,41.11,371,23.24,0.51


In [4]:
#Extract all foods labeled "CANDIES" from dataset
candy_df = split_df.loc[split_df[0] == 'CANDIES']

#Drop unnecessary columns
new_candy_df = candy_df[[1, 2, 'NDB_No', 'Water_(g)', 'Energ_Kcal', 'Protein_(g)', 'Sugar_Tot_(g)']]

#Save new csv
new_candy_df.to_csv('candy.csv')

new_candy_df

,1,2,NDB_No,Water_(g),Energ_Kcal,Protein_(g),Sugar_Tot_(g)
6049,TOBLERONE,MILK CHOC W/ HONEY & ALMOND NOUGAT,19001,3.28,525,5.71,55.74
6068,HONEY-COMBED,W/ PNUT BUTTER,19025,2.54,486,8.72,62.68
6100,TOOTSIE ROLL,CHOCOLATE-FLAVOR ROLL,19064,6.69,387,1.59,56.32
6101,ALMOND JOY CANDY BAR,None,19065,8.20,479,4.13,48.34
6102,TWIZZLERS CHERRY BITES,None,19067,15.00,338,2.97,NaN
...,...,...,...,...,...,...,...
8649,CHOC COVERED,CARAMEL W/NUTS,43031,6.10,470,9.50,41.35
8650,NOUGAT,W/ ALMONDS,43046,2.15,398,3.33,44.97
8651,GUM DROPS,NO SUGAR OR LO CAL (SORBITOL),43057,11.70,354,0.00,70.00
8652,HARD,DIETETIC OR LO CAL (SORBITOL),43058,1.40,394,0.00,0.00


# Transform

In Excel merged candy_detail(column "2") with candy_names(column "1") where column "1" showed brand name only. 

In Excel match "candy_names" from candy.csv to "competitorname" in candy_rankings.csv.

In [5]:
#Import csv with matched names
csv_file = "./Resources/transformed_candy_3.csv"
clean_candy_df = pd.read_csv(csv_file, encoding = 'utf8')

#TRANSFORM: Removed foreign characters from candy_names
clean_candy_df["candy_names"]=clean_candy_df["candy_names"].str.replace("Õ","")
clean_candy_df

,usda_number,candy_names,candy_detail,NDB_No,Water_(g),Energ_Kcal,Protein_(g),Sugar_Tot_(g)
0,6109,CARAMELS,CHOCOLATE-FLAVOR ROLL,19076,6.69,387,1.59,56.32
1,6174,100 Grand,NaN,19144,6.10,468,2.50,51.90
2,6188,3 Musketeers,NaN,19159,5.80,436,2.60,66.89
3,6130,5TH AVENUE CANDY BAR,NaN,19098,2.30,482,8.78,47.19
4,6183,AFTER EIGHT MINTS,NaN,19153,6.30,432,1.67,66.60
...,...,...,...,...,...,...,...,...
129,6144,Twizzlers,NaN,19112,15.00,348,2.56,39.64
130,6191,WHATCHAMACALLIT CANDY BAR,NaN,19162,3.04,494,8.04,48.86
131,6119,WHITE CHOC,NaN,19087,1.30,539,5.87,59.00
132,6205,YORK BITES,NaN,19181,9.09,394,1.78,75.00


In [14]:
#TRANSFORM: Change columns names
clean_candy_df.rename(columns={"candy_names":"competitorname", "Water_(g)":"water_g", "Protein_(g)":"protein_g", "Sugar_Tot_(g)":"sugar_g", "Energ_Kcal":"energy_kcal", "NDB_No":"ndb_no"}, inplace=True)
clean_candy_df

,usda_number,competitorname,candy_detail,ndb_no,water_g,energy_kcal,protein_g,sugar_g
0,6109,CARAMELS,CHOCOLATE-FLAVOR ROLL,19076,6.69,387,1.59,56.32
1,6174,100 Grand,NaN,19144,6.10,468,2.50,51.90
2,6188,3 Musketeers,NaN,19159,5.80,436,2.60,66.89
3,6130,5TH AVENUE CANDY BAR,NaN,19098,2.30,482,8.78,47.19
4,6183,AFTER EIGHT MINTS,NaN,19153,6.30,432,1.67,66.60
...,...,...,...,...,...,...,...,...
129,6144,Twizzlers,NaN,19112,15.00,348,2.56,39.64
130,6191,WHATCHAMACALLIT CANDY BAR,NaN,19162,3.04,494,8.04,48.86
131,6119,WHITE CHOC,NaN,19087,1.30,539,5.87,59.00
132,6205,YORK BITES,NaN,19181,9.09,394,1.78,75.00


In [15]:
# Import candy_ranking dataset from FiveThirtyEight
csv_file = "./Resources/candy_ranking.csv"
candy_rank_df = pd.read_csv(csv_file, encoding = 'utf8')

#TRANSFORM: Remove foreiogn characters
candy_rank_df["competitorname"]=candy_rank_df["competitorname"].str.replace("Õ","")
candy_rank_df

,competitorname,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent
0,100 Grand,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725
1,3 Musketeers,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936
2,One dime,0,0,0,0,0,0,0,0,0,0.011,0.116,32.261086
3,One quarter,0,0,0,0,0,0,0,0,0,0.011,0.511,46.116505
4,Air Heads,0,1,0,0,0,0,0,0,0,0.906,0.511,52.341465
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Twizzlers,0,1,0,0,0,0,0,0,0,0.220,0.116,45.466282
81,Warheads,0,1,0,0,0,0,1,0,0,0.093,0.116,39.011898
82,Welchs Fruit Snacks,0,1,0,0,0,0,0,0,1,0.313,0.313,44.375519
83,Werthers Original Caramel,0,0,1,0,0,0,1,0,0,0.186,0.267,41.904308


In [16]:
#TRANSFORM: Merge datasets on "competitorname" and drop any rows that don't match
merge_candy_df = pd.merge(clean_candy_df, candy_rank_df, on="competitorname", how="inner")
merge_candy_df

,usda_number,competitorname,candy_detail,ndb_no,water_g,energy_kcal,protein_g,sugar_g,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent
0,6174,100 Grand,NaN,19144,6.10,468,2.50,51.90,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725
1,6188,3 Musketeers,NaN,19159,5.80,436,2.60,66.89,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936
2,6101,Almond Joy,NaN,19065,8.20,479,4.13,48.34,1,0,0,1,0,0,0,1,0,0.465,0.767,50.347546
3,6143,Baby Ruth,NaN,19111,6.99,459,5.40,54.00,1,0,1,1,1,0,0,1,0,0.604,0.767,56.914547
4,6104,Nestle Butterfinger,NaN,19069,1.23,459,5.40,45.90,1,0,0,1,0,0,0,1,0,0.604,0.767,70.735641
5,6141,Kit Kat,NaN,19109,1.63,518,6.51,48.68,1,0,0,0,0,1,0,1,0,0.313,0.511,76.768600
6,6142,Hersheys Krackel,NaN,19110,1.17,512,6.62,52.53,1,0,0,0,0,1,0,1,0,0.430,0.918,62.284481
7,6171,M&Ms,NaN,19141,1.70,492,4.33,63.68,1,0,0,0,0,0,0,0,1,0.825,0.651,66.574585
8,8579,Peanut butter M&Ms,NaN,42148,2.00,529,10.16,47.15,1,0,0,1,0,0,0,0,1,0.825,0.651,71.465050
9,6170,Peanut M&Ms,NaN,19140,2.48,515,9.57,50.75,1,0,0,1,0,0,0,0,1,0.593,0.651,69.483788


In [17]:
#TRANSFORM: Drop irrelevant columns
final_candy_df = merge_candy_df[["competitorname", 'ndb_no', 'water_g', 'energy_kcal', 'protein_g', 'sugar_g', "chocolate", "fruity", "caramel", "peanutyalmondy","crispedricewafer","hard","bar", "pluribus", "sugarpercent", "pricepercent","winpercent"]]
final_candy_df

,competitorname,ndb_no,water_g,energy_kcal,protein_g,sugar_g,chocolate,fruity,caramel,peanutyalmondy,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent
0,100 Grand,19144,6.10,468,2.50,51.90,1,0,1,0,1,0,1,0,0.732,0.860,66.971725
1,3 Musketeers,19159,5.80,436,2.60,66.89,1,0,0,0,0,0,1,0,0.604,0.511,67.602936
2,Almond Joy,19065,8.20,479,4.13,48.34,1,0,0,1,0,0,1,0,0.465,0.767,50.347546
3,Baby Ruth,19111,6.99,459,5.40,54.00,1,0,1,1,0,0,1,0,0.604,0.767,56.914547
4,Nestle Butterfinger,19069,1.23,459,5.40,45.90,1,0,0,1,0,0,1,0,0.604,0.767,70.735641
5,Kit Kat,19109,1.63,518,6.51,48.68,1,0,0,0,1,0,1,0,0.313,0.511,76.768600
6,Hersheys Krackel,19110,1.17,512,6.62,52.53,1,0,0,0,1,0,1,0,0.430,0.918,62.284481
7,M&Ms,19141,1.70,492,4.33,63.68,1,0,0,0,0,0,0,1,0.825,0.651,66.574585
8,Peanut butter M&Ms,42148,2.00,529,10.16,47.15,1,0,0,1,0,0,0,1,0.825,0.651,71.465050
9,Peanut M&Ms,19140,2.48,515,9.57,50.75,1,0,0,1,0,0,0,1,0.593,0.651,69.483788


In [18]:
#Drop duplicates
final_candy_df.drop_duplicates(subset ="competitorname", 
                     keep ='first', inplace = True)
final_candy_df

C:\Users\emmam\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,competitorname,ndb_no,water_g,energy_kcal,protein_g,sugar_g,chocolate,fruity,caramel,peanutyalmondy,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent
0,100 Grand,19144,6.10,468,2.50,51.90,1,0,1,0,1,0,1,0,0.732,0.860,66.971725
1,3 Musketeers,19159,5.80,436,2.60,66.89,1,0,0,0,0,0,1,0,0.604,0.511,67.602936
2,Almond Joy,19065,8.20,479,4.13,48.34,1,0,0,1,0,0,1,0,0.465,0.767,50.347546
3,Baby Ruth,19111,6.99,459,5.40,54.00,1,0,1,1,0,0,1,0,0.604,0.767,56.914547
4,Nestle Butterfinger,19069,1.23,459,5.40,45.90,1,0,0,1,0,0,1,0,0.604,0.767,70.735641
5,Kit Kat,19109,1.63,518,6.51,48.68,1,0,0,0,1,0,1,0,0.313,0.511,76.768600
6,Hersheys Krackel,19110,1.17,512,6.62,52.53,1,0,0,0,1,0,1,0,0.430,0.918,62.284481
7,M&Ms,19141,1.70,492,4.33,63.68,1,0,0,0,0,0,0,1,0.825,0.651,66.574585
8,Peanut butter M&Ms,42148,2.00,529,10.16,47.15,1,0,0,1,0,0,0,1,0.825,0.651,71.465050
9,Peanut M&Ms,19140,2.48,515,9.57,50.75,1,0,0,1,0,0,0,1,0.593,0.651,69.483788


# Load

Create database connection to relational database in PostgreSQL

In [19]:

connection_string = "postgres:<password>@localhost:5432/candy_db"
engine = create_engine(f'postgresql://{connection_string}')

In [20]:
#Confirmed tables
engine.table_names()

['candy']

In [21]:
#LOAD: Load datafram into database
final_candy_df.to_sql(name='candy', con=engine, if_exists='append', index=True, index_label="id")